# Jupyterのマジックコマンドを利用して、簡単なアプリを作る

# これなに?

Fringe81で機械学習エンジニアをやっている長谷川です。  業務では主に広告配信のサーバ側の開発(Scala)および、その配信上で実行する機械学習アルゴリズムの開発(Python)をやっています。  
この記事では、Jupyterのマジックコマンドを利用して複数の言語を組み合わせることで、簡単なアプリケーション作成方法について書きます。   
なおjsのコードが少し出てきますが、、僕は殆ど書いたことがないので、なにか間違いがありましたらごめんなさい。  

## 作るアプリの実行例
twitterのデータを手動でラベルづけするアプリです。Jupter上で実行してます。

TODO

# Jupyterとは?

さまざまなコードのインタラクティブな実行などが可能になる、ブラウザベースのエディタです。余談ですがこの記事もすべてJupyter上で書きました。  
pythonのイメージが強いですが、kernelさえ入れ替えれば大抵の言語は実行可能です。(ちなみにJupyterの名前の由来はJulia+python+Rだそうです。)  


# マジックコマンドとは?

さらに、juypterには**マジックコマンド**と呼ばれる機能があり、セルの先頭に*%%xxx*のようなアノテーションを付けると、シェルやsqlなど、さまざまな処理が実行可能になります。  
そのマジックコマンドを利用し、pythonのコードと**%%javascriptと%%htmlを組み合わせることにより、簡易的なラベルで付けアプリを作成してみます。**

## %%javascript

In [2]:
%%javascript
alert('こんにちはJS');

<IPython.core.display.Javascript object>

Jupyter上で実行するとダイアログができます。

## %HTML

In [8]:
%%html
<script>
alert("こんにちは　HTML");
</script>

同じくダイアログが出ます。

# ライブラリのバージョン

TODO

# アプリの例: データセットのラベルづけ
機械学習をする上で、ある意味アルゴリズム以上に辛いのは、データセットの作成です。教師あり学習の場合、当然教師ラベルを付与する必要があるのですが、そんな都合よくラベルが付いているデータがあることは少なく、仮についていたとしても人力での補正が必要な場合は多々あります。  
そこで、人力でのラベル付をJupyter上で完結する形で実装してみます。
例として、twitterから某メーカーのきのこたけのこに関して検索したデータを取得し、tweetの内容をみて、たけのこ派かどうかのラベル付けを行います。 (たけのこ派を1、そうでないかどちらとも言えない場合0とする)^1   
なお、下記の例(特にjavascriptの部分)はこちらの、[Python Real World Data Science ](https://www.packtpub.com/big-data-and-business-intelligence/python-real-world-data-science)をかなり参考にしました。(邦訳は多分ないですが、とてもいい本です)

## Tokenのロード
まず、別ファイルに記述してあるTokenを読み込みます。  
機密情報をJupyeにべた書きすると、予期せぬところで流出するリスクが極めて高いので、別ファイルに書き出しておいて読み込んだほうが良いです。

```
# twitter.json
{
"consumer_key":"xxx",
"consumer_secret":"xxx",
"token":"xxx",
"token_secret":"xxx"
}
```

In [3]:
# twitterのAPIを叩くためにpipでいれる
!pip install twitter

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import twitter
import json 

# confidentailな情報をうっかりgitにあげないように、別ファイルに保存してあるものを取得
with open('./twitter.json','r') as file: 
    token=json.load(file)
authorization = twitter.OAuth(**token)

# twitterとのI/Fのためのオブジェクト
t=twitter.Twitter(auth=authorization)

## APIを叩く
データを取得するためのメソッドです。tweetが存在するもののみを返します。

In [6]:
tweets=[] # tweetを格納していくlist

def  twitter_gen(search_word,count=100):
    search_results=t.search.tweets(q=search_word,count=count)['statuses']
    index=0
    
    while True:
        search_result=search_results[index]
        if 'text' in search_result: # contentにtextがあるもののみを取得
            tweet=search_result['text']
            tweets.append(tweet)           
            yield tweet
        index+=1 


In [46]:
tweet_meiji_war=twitter_gen('たけのこ派 きのこ派')

generatorで取得しているので、`next`で読み込みます

In [50]:
print(next(tweet_meiji_war))

笹谷「卒業シーズンとかさ、『いっぱい喧嘩もしたけど、だからこそ今がある』みたいなキャッチコピーよく聞くよな」茂庭「鎌ちはよく二口に突っかかってるし、当てはまるんじゃない？」鎌先「この間はきのこ派たけのこ派で戦争したけど」茂庭「あーそれだめなやつだわ」笹谷「つか戦争って…」


取得できました。

## アプリケーションの動きをJSで実装
アプリケーション上で実行するために、**マジックコマンドを利用して、jsでscriptを記述します。**  
そして、そのscriptから**Jupyterのkernelに処理を送る**ことで、バックグラウンドでpythonを実行し、データを読み込みます。

In [14]:
# 正解ラベルを格納するlist
labels=[]

まず、ユーザーから判定されたラベルを追加していく処理です。  
Jupyterには`Python.notebook.kernel`を使用することで、処理が送信できます。  
このオブジェクトの`execute`メソッドにより、JSからpythonのスクリプトを実行できます。

In [15]:
%%javascript

function set_label(label){
    var kernel = IPython.notebook.kernel;
    kernel.execute("labels.append(" + label + ")");
    load_next_tweet();
}


<IPython.core.display.Javascript object>

TODO

In [54]:
%%javascript
function load_next_tweet(){
   var code_input = "next(tweet_meiji_war)";
   var kernel = IPython.notebook.kernel;
   var callbacks = { 'iopub' : {'output' : handle_output}};
   kernel.execute(code_input, callbacks, {silent:false});
}

<IPython.core.display.Javascript object>

In [17]:
%%javascript

function handle_output(out){
   var res = out.content.data["text/plain"];
   $("div#tweet_text").html(res);
}

<IPython.core.display.Javascript object>

TODO

In [55]:
%%html
<div name="tweetbox">
    たけのこ派の場合は1、そうでない場合は0を押してください<br>
Tweet: 
    <div id="tweet_text" value="text"></div><br>
<input type=text id="capture"></input><br>
</div>



<script>
load_next_tweet();
$("input#capture").keypress(function(e) {
if(e.which == 48) {
    set_label(0);
    $("input#capture").val("");
}else if (e.which==49){
    set_label(1);
    $("input#capture").val("");
  }
});

function set_label(label){
    var kernel = IPython.notebook.kernel;
    kernel.execute("labels.append(" + label + ")");
    load_next_tweet();
}

function load_next_tweet(){
   var code_input = "next(tweet_meiji_war)";
   var kernel = IPython.notebook.kernel;
   var callbacks = { 'iopub' : {'output' : handle_output}};
   kernel.execute(code_input, callbacks, {silent:false});
}

function handle_output(out){
   var res = out.content.data["text/plain"];
   $("div#tweet_text").html(res);
}
</script>


## ラベルを確認

In [53]:
print(labels)
print(tweets)

[0, 1, 1]
['技術系のポッドキャスト教えてくりーーー / “エンジニアが情報収集とトレンドを追うのに読むと良いサイト - Qiita” https://t.co/VNjwy5Uo6a', 'JavaScript Standard Styleのススメ - Qiita https://t.co/cXSBK5d4vR', 'RT @sugitk: きた〜\n\nOpenShift 全部俺 Advent Calendar 2018 #Qiita https://t.co/kVKfc66GhD', 'Cloud Datastoreのクエリでがんばるハナシ [Go] on @Qiita https://t.co/AkdpyPZagj', '“文末のセミコロンは禁止” / “JavaScript Standard Styleのススメ - Qiita” https://t.co/D4yyvfl7OY', 'RT @koukiwf: やべかった。//Unity でリアルな雲を表現するためのシェーダを作成する https://t.co/4JG2hz7QUF', 'RT @drken1215: クォータニオンについて総合的に総整理した記事を書いてみました！\nUnity、3D CG、航空宇宙と分野横断的に活躍しているのが心そそられます。\n\n昨日の夜 12 時に、突然編集画面が真っ白になったときは泣きたくなったん...^^;\n(昨日朝時点のバ…', '@H_O_R_Z https://t.co/SeVvv7XWOo\nhttps://t.co/82x283s2Wi', '創作キャラの、きのこたけのこ戦争何派か知りたくないです？私は知りたい', '笹谷「卒業シーズンとかさ、『いっぱい喧嘩もしたけど、だからこそ今がある』みたいなキャッチコピーよく聞くよな」茂庭「鎌ちはよく二口に突っかかってるし、当てはまるんじゃない？」鎌先「この間はきのこ派たけのこ派で戦争したけど」茂庭「あーそれだめなやつだわ」笹谷「つか戦争って…」', '今忙しくて、後にしてくれませんか！いや、きのこたけのこ戦争をしてて。ザップさんと。', 'な、なんか最近きのこたけのこ戦争起きそうになってるんだけど…(￣▽￣;)', '風見さんときのこvsたけのこで戦争した後、こんどは粒あんvsこしあんで仁義なき戦いを繰り

無事、入力したラベルが入っています。

## 保存してデータセットを作成
最後にデータセットをファイルに書き出しましょう。

In [5]:
import pandas as pd
pd.DataFrame(zip())

TypeError: data argument can't be an iterator

^1 例が無意味にセンシティブな内容ですが、正例、負例ちょうどよいバランスで取得できるデータが他に思いつかなかったので、これにしました。